In [102]:
import pandas as pd
import pyterrier as pt
import yaml
import os
from src.load_index import load_index, load_topics, load_qrels, tag
from src.extend_runs import extend_run_full
import sqlite3
from repro_eval.Evaluator import RpdEvaluator
import pytrec_eval

from repro_eval.util import arp, arp_scores

if not pt.started():
    pt.init()

In [2]:
with open("data/LongEval/metadata.yml", "r") as yamlfile:
    config = yaml.load(yamlfile, Loader=yaml.FullLoader)

results_path = "data/results/filter_not_rel/"

index = load_index("t3")
topics = load_topics("t3", "test")

>>> Loaded index with  2049729 documents.


In [6]:
BM25 = pt.BatchRetrieve(index, wmodel="BM25", verbose=True, num_results=1500)

pt.io.write_results(
    BM25(topics), results_path + f"/CIR_BM25_D-t3_T-t3-long"
)

BR(BM25): 100%|██████████| 599/599 [08:29<00:00,  1.18q/s]


In [3]:
extend_run_full(results_path + f"CIR_BM25_D-t3_T-t3-long")

>>> Loaded run
>>> Load doc map


34it [04:25,  7.81s/it]                             


>>> Load query map
>>> Extend run


In [172]:
run = pd.read_csv(results_path + f"CIR_BM25_D-t3_T-t3-l_extended.t3", sep=" ")

/var/folders/pz/f66p4s0s11z2c1c1l3v0qbh40000gn/T/ipykernel_17639/3447229843.py:1: DtypeWarning: Columns (13) have mixed types. Specify dtype option on import or set low_memory=False.
  run = pd.read_csv(results_path + f"CIR_BM25_D-t3_T-t3-l_extended.t3", sep=" ")


In [173]:
conn = sqlite3.connect("data/database.db")
query = "SELECT * FROM qrel"
qrels = pd.read_sql_query(query, conn)

In [174]:
qrels["key"] = qrels["queryid"]+qrels["docid"]

In [175]:
qmap=qrels[["key", "relevance"]].set_index("key").to_dict()["relevance"]

In [176]:
def get_qrel(row, subcollection):
    query_id = row[f"queryid_{subcollection}"]
    doc_id = row[f"docid_{subcollection}"]
    if isinstance(query_id, str) and isinstance(doc_id, str):
        return qmap.get(row[f"queryid_{subcollection}"] + row[f"docid_{subcollection}"], None)
    else:
        return None
       
for subcollection in ["t0", "t1", "t2"]:
    run[f"qrel_{subcollection}"] = run.apply(get_qrel, subcollection=subcollection, axis=1)


# Filter not rel

In [127]:
reranked = run.copy()

In [128]:
for subcollection in ["t0", "t1", "t2"]:
    reranked = reranked[reranked[f"qrel_{subcollection}"] != 0.0]

In [129]:
reranked = reranked.sort_values(['queryid','score'], ascending=False).groupby('queryid').head(1000)
reranked['rank'] = reranked.groupby('queryid')['score'].rank(ascending=False).astype(int)


In [130]:
run_name = f"CIR_BM25_D-t3_T-t3-rrnotrel"
run_reranked_path = os.path.join(results_path, run_name)

In [131]:
pt.io.write_results(reranked[["queryid", "0", "docid", "score", "rank", "run"]].rename(columns={"queryid": "qid", "docid": "docno"}), run_reranked_path, format='trec', run_name=run_name)

In [132]:
base_path = "data"

with open(os.path.join(base_path, config["subcollections"]["t3"]["qrels"]["test"]), "r") as f_qrels:
    qrels = pytrec_eval.parse_qrel(f_qrels)
evaluator = pytrec_eval.RelevanceEvaluator(qrels, pytrec_eval.supported_measures)

In [133]:
with open(run_reranked_path) as run_reranked:
    r = pytrec_eval.parse_run(run_reranked)
    scores = evaluator.evaluate(r)
    print(arp_scores(scores)["P_10"])
    print(arp_scores(scores)["bpref"])
    print(arp_scores(scores)["ndcg"])

0.15953177257525084
0.4414876050518121
0.3585802860890029


# Boostrel

In [189]:
reranked = run.copy()

In [190]:
for subcollection in ["t0", "t1", "t2"]:
    # Modify values in the subset
    reranked.loc[reranked[f"qrel_{subcollection}"] == 2, 'score'] = reranked.loc[reranked[f"qrel_{subcollection}"] == 2, 'score'] * 2
    

    # Modify values in the subset
    reranked.loc[reranked[f"qrel_{subcollection}"] == 1, 'score'] = reranked.loc[reranked[f"qrel_{subcollection}"] == 1.5, 'score'] * 1.5


In [181]:
reranked = reranked.sort_values(['queryid','score'], ascending=False).groupby('queryid').head(1000)
#reranked['rank'] = reranked.groupby('queryid')['score'].rank(ascending=False).astype(int)

In [188]:
reranked.groupby('queryid')['score'].rank(ascending=False).astype(int)

IntCastingNaNError: Cannot convert non-finite values (NA or inf) to integer

In [186]:
reranked

,queryid,0,docid,relevance,score,run,docid_t0,docid_t1,docid_t2,docid_t3,...,docid_t5,queryid_t0,queryid_t1,queryid_t2,queryid_t3,queryid_t4,queryid_t5,qrel_t0,qrel_t1,qrel_t2
1500,q012396,Q0,doc012304304522,0,17.976831,pyterrier,NaN,NaN,NaN,doc012304304522,...,NaN,NaN,NaN,NaN,q012396,NaN,NaN,NaN,NaN,NaN
1501,q012396,Q0,doc012310909638,1,17.835378,pyterrier,NaN,NaN,NaN,doc012310909638,...,doc082304309198,NaN,NaN,NaN,q012396,NaN,NaN,NaN,NaN,NaN
1502,q012396,Q0,doc012306711017,2,17.547611,pyterrier,doc062200705775,doc072203700443,doc092203203752,doc012306711017,...,doc082306914409,NaN,NaN,NaN,q012396,NaN,NaN,NaN,NaN,NaN
1503,q012396,Q0,doc012312210606,3,17.514200,pyterrier,NaN,NaN,NaN,doc012312210606,...,NaN,NaN,NaN,NaN,q012396,NaN,NaN,NaN,NaN,NaN
1504,q012396,Q0,doc012305915202,4,17.474759,pyterrier,NaN,NaN,NaN,doc012305915202,...,doc082311201967,NaN,NaN,NaN,q012396,NaN,NaN,NaN,NaN,NaN
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
634607,q0123103079215124,Q0,doc012304213581,995,12.971478,pyterrier,NaN,NaN,NaN,doc012304213581,...,doc082302813260,NaN,NaN,NaN,q0123103079215124,NaN,NaN,NaN,NaN,NaN
634608,q0123103079215124,Q0,doc012309500215,996,12.971478,pyterrier,NaN,NaN,NaN,doc012309500215,...,doc082309212385,NaN,NaN,NaN,q0123103079215124,NaN,NaN,NaN,NaN,NaN
634609,q0123103079215124,Q0,doc012309405964,997,12.971478,pyterrier,NaN,NaN,NaN,doc012309405964,...,doc082304316860,NaN,NaN,NaN,q0123103079215124,NaN,NaN,NaN,NaN,NaN
634610,q0123103079215124,Q0,doc012309406124,998,12.971478,pyterrier,NaN,NaN,NaN,doc012309406124,...,doc082302703373,NaN,NaN,NaN,q0123103079215124,NaN,NaN,NaN,NaN,NaN


In [184]:
run_name = f"CIR_BM25_D-t3_T-t3-rrrel"
run_reranked_path = os.path.join(results_path, run_name)

In [185]:
pt.io.write_results(reranked[["queryid", "0", "docid", "score", "rank", "run"]].rename(columns={"queryid": "qid", "docid": "docno"}), run_reranked_path, format='trec', run_name=run_name)

base_path = "data"

with open(os.path.join(base_path, config["subcollections"]["t3"]["qrels"]["test"]), "r") as f_qrels:
    qrels = pytrec_eval.parse_qrel(f_qrels)
evaluator = pytrec_eval.RelevanceEvaluator(qrels, pytrec_eval.supported_measures)

KeyError: "['rank'] not in index"